In [ ]:
import pandas as pd
import os
import re

In [ ]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [ ]:
PASTA_DE_DADOS = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_download']
PASTA_SAIDA = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['caminho_arquivo_convertido']

In [ ]:
patternPeriodo = re.compile('(.*)T(.*).csv')

In [ ]:
lista_arquivos = os.listdir(PASTA_DE_DADOS)

In [ ]:
if (not(os.path.exists(PASTA_SAIDA))):
    os.mkdir(PASTA_SAIDA)

In [ ]:
for _idx,arquivo in enumerate(lista_arquivos):
    _match = patternPeriodo.findall(arquivo)[0]
    _semestre = _match[0]
    _ano =  _match[1]
    df = pd.read_csv(PASTA_DE_DADOS+arquivo,sep=";",dtype=str,encoding="ISO-8859-1")
    df.insert(0,'SEMESTRE',_semestre)
    df.insert(0,'ANO',_ano)
    df['VL_SALDO_FINAL'] = df['VL_SALDO_FINAL'].str.replace(",",".").astype(float)
    df.drop(columns="DATA",inplace=True)
    df_saida = df[['ANO', 'SEMESTRE','REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO','VL_SALDO_FINAL']]
    df_saida.to_csv(PASTA_SAIDA+arquivo,index=False)
    print("Convetido ",(_idx+1),"de",len(lista_arquivos))